In [ ]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
import tensorflow
from tensorflow import keras
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
start = '2010-01-01'
end = '2019-12-31'

data = web.DataReader('TSLA', 'yahoo', start, end)
df = pd.DataFrame(data)
df.head()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()

In [ ]:
plt.plot(df.Close)

In [ ]:
ma100 = df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize = (12, 6))
plt.plot(df.Close)
plt.plot(ma100)
plt.show()

In [ ]:
ma200 = df.Close.rolling(200).mean()
ma200

In [ ]:
plt.figure(figsize = (12, 6))
plt.plot(df.Close)
plt.plot(ma100, color = 'red')
plt.plot(ma200, color = 'green')
plt.show()

In [ ]:
df.shape

In [ ]:
dataTraining = pd.DataFrame(df['Close'][0 : int(len(df) * 0.70)])
dataTesting = pd.DataFrame(df['Close'][int(len(df) * 0.70) : int(len(df))])

print(dataTraining.shape)
print(dataTesting.shape)

In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))

dataTraining = scaler.fit_transform(dataTraining)
dataTraining

In [ ]:
xTrain = []
yTrain = []

for i in range(100, dataTraining.shape[0]):
  xTrain.append(dataTraining[i - 100 : i])
  yTrain.append(dataTraining[i, 0])

xTrain, yTrain = np.array(xTrain), np.array(yTrain)

In [ ]:
xTrain.shape

In [ ]:
model = Sequential()

model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (xTrain.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))
model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(xTrain, yTrain, epochs = 50)

In [ ]:
model.save('Stock model.h5')

In [ ]:
past100Days = dataTesting.tail(100)
totalDf = past100Days.append(dataTesting, ignore_index = True)

inputData = scaler.fit_transform(totalDf)
inputData.shape

In [ ]:
xTest = []
yTest = []

for i in range(100, inputData.shape[0]):
  xTest.append(inputData[i - 100 : i])
  yTest.append(inputData[i, 0])

xTest, yTest = np.array(xTest), np.array(yTest)

In [ ]:
xTest.shape

In [ ]:
yPred = model.predict(xTest)
yPred.shape

In [ ]:
yPred

In [ ]:
yTest

In [ ]:
scaler.scale_

In [ ]:
scaleFactor = 1 / 0.01984363

yPred = yPred * scaleFactor
yTest = yTest * scaleFactor

In [ ]:
plt.figure(figsize = (15, 8))
plt.plot(yTest, color = 'blue', label = 'Actual Price')
plt.plot(yPred, color = 'red', label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()